# 00B — Logistische Regressie als Brug naar Neural Nets

Waarom dit notebook?  
Logistische regressie is vaak de meest begrijpelijke stap tussen simpele ML en neural networks.

Wat je leert:
1. Hoe logistische regressie kansen voorspelt
2. Waarom train/validation/test split belangrijk is
3. Hoe je modelkeuzes maakt met validation
4. Hoe dit conceptueel lijkt op een neuron in een neural net

## Intuïtie: van score naar kans

Logistische regressie maakt eerst een lineaire score en zet die met de **sigmoid-functie** om naar een kans tussen 0 en 1.

Met een drempel (bijv. 0.5) maak je daarna de beslissing: churn of geen churn.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score, recall_score

np.random.seed(42)

In [ ]:
x = np.linspace(-10, 10, 200)
sigmoid = 1 / (1 + np.exp(-x))

plt.figure(figsize=(7, 4))
plt.plot(x, sigmoid)
plt.title('Sigmoid-functie: zet score om naar kans')
plt.xlabel('Lineaire score')
plt.ylabel('Kans')
plt.grid(alpha=0.3)
plt.show()

## Data laden en voorbereiden (telecom churn)

In [ ]:
df = pd.read_csv('../data/churn-bigml-20.csv')
df['Churn'] = df['Churn'].astype(int)

categorical_cols = df.select_dtypes(include='object').columns
df_encoded = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

X = df_encoded.drop(columns='Churn')
y = df_encoded['Churn']

X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.40, random_state=42, stratify=y
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.50, random_state=42, stratify=y_temp
)

print(f'Train: {len(X_train)}')
print(f'Validation: {len(X_val)}')
print(f'Test: {len(X_test)}')

In [ ]:
labels = ['Train', 'Validation', 'Test']
sizes = [len(X_train), len(X_val), len(X_test)]

fig, axes = plt.subplots(1, 2, figsize=(11, 4))
axes[0].bar(labels, sizes, color=['steelblue', 'orange', 'seagreen'])
axes[0].set_title('Split-groottes')
axes[0].set_ylabel('Aantal rijen')

axes[1].pie(sizes, labels=labels, autopct='%1.0f%%', colors=['steelblue', 'orange', 'seagreen'])
axes[1].set_title('Split-verhouding')

plt.tight_layout()
plt.show()

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

## Hyperparameter kiezen met validation-set

We variëren de regularisatie-sterkte `C`. Hoger `C` = minder strenge regularisatie.

In [ ]:
c_waarden = [0.01, 0.1, 1, 10, 100]
val_scores = []

for c in c_waarden:
    lr = LogisticRegression(C=c, max_iter=2000, random_state=42)
    lr.fit(X_train_scaled, y_train)
    val_pred = lr.predict(X_val_scaled)
    val_scores.append(accuracy_score(y_val, val_pred))

beste_index = int(np.argmax(val_scores))
beste_c = c_waarden[beste_index]
print(f'Beste C op validation-set: {beste_c}')

plt.figure(figsize=(7, 4))
plt.plot(c_waarden, val_scores, marker='o')
plt.xscale('log')
plt.title('Validation accuracy per C')
plt.xlabel('C (log-schaal)')
plt.ylabel('Accuracy')
plt.grid(alpha=0.3)
plt.show()

In [ ]:
final_model = LogisticRegression(C=beste_c, max_iter=2000, random_state=42)
final_model.fit(X_train_scaled, y_train)

test_pred = final_model.predict(X_test_scaled)
test_proba = final_model.predict_proba(X_test_scaled)[:, 1]

print(f'Test accuracy: {accuracy_score(y_test, test_pred):.3f}')
print()
print(classification_report(y_test, test_pred, target_names=['Geen churn', 'Wel churn']))

cm = confusion_matrix(y_test, test_pred)
plt.figure(figsize=(5, 4))
plt.imshow(cm, cmap='Purples')
plt.title('Confusion matrix (test-set)')
plt.xlabel('Voorspeld')
plt.ylabel('Werkelijk')
plt.xticks([0, 1], ['Geen churn', 'Wel churn'])
plt.yticks([0, 1], ['Geen churn', 'Wel churn'])
for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        plt.text(j, i, cm[i, j], ha='center', va='center')
plt.colorbar()
plt.tight_layout()
plt.show()

### Uitleg Confusion Matrix (wat betekent elk getal?)

Lees de matrix altijd zo:

- **Rijen = werkelijkheid** (wat er echt gebeurde).
- **Kolommen = voorspelling** (wat het model zei).

Per vak:

- **Linksboven**: correct voorspeld als **geen churn**.
- **Rechtsboven**: model voorspelt **wel churn**, maar klant bleef (**vals alarm**).
- **Linksonder**: model voorspelt **geen churn**, maar klant vertrok (**gemiste churner**).
- **Rechtsonder**: correct voorspeld als **wel churn**.

Business-vertaling:

- Als gemiste churners duur zijn, wil je vooral het **linksonder vak** kleiner maken.
- Als onnodig klantcontact duur is, wil je vooral het **rechtsboven vak** kleiner maken.

In [ ]:
coef_df = pd.DataFrame({
    'feature': X.columns,
    'coefficient': final_model.coef_[0]
})

top = coef_df.reindex(coef_df['coefficient'].abs().sort_values(ascending=False).index).head(10)

plt.figure(figsize=(9, 5))
plt.barh(top['feature'][::-1], top['coefficient'][::-1], color='teal')
plt.title('Top 10 meest invloedrijke kenmerken (logistische regressie)')
plt.xlabel('Coefficient (richting + sterkte)')
plt.tight_layout()
plt.show()

top

In [ ]:
thresholds = np.arange(0.1, 0.91, 0.1)
precisions, recalls = [], []

for t in thresholds:
    pred_t = (test_proba >= t).astype(int)
    precisions.append(precision_score(y_test, pred_t, zero_division=0))
    recalls.append(recall_score(y_test, pred_t, zero_division=0))

plt.figure(figsize=(8, 4))
plt.plot(thresholds, precisions, marker='o', label='Precision')
plt.plot(thresholds, recalls, marker='o', label='Recall')
plt.title('Effect van drempel op precision en recall')
plt.xlabel('Drempel')
plt.ylabel('Score')
plt.ylim(0, 1)
plt.grid(alpha=0.3)
plt.legend()
plt.show()

## Koppeling naar neural nets

Een enkel neuron met sigmoid-activatie lijkt sterk op logistische regressie.

Verschil:
- Logistische regressie: 1 lineaire stap + sigmoid
- Neural net: veel neuronen + meerdere lagen, waardoor complexere patronen mogelijk zijn

Daarmee is logistische regressie een perfecte tussenstap vóór een eerste neural network.